# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [12]:
# import libraries
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
import os

In [13]:
# load messages dataset
messages = pd.read_csv('Data/messages.csv')
messages.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [14]:
# load categories dataset
categories = pd.read_csv('Data/categories.csv')
categories.head()

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [15]:
# Merge datasets on 'id'
df = messages.merge(categories, on='id')

df.head(5)

,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...


### 3. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.

In [16]:
# Split the values in the `categories` column on the `;` character
categories_split = df['categories'].str.split(';', expand=True)

# Use the first row of categories_split dataframe to create column names for the categories data
row = categories_split.iloc[0]

# Extract a list of new column names for categories
category_colnames = row.apply(lambda x: x[:-2])

# Rename columns of `categories_split` with new column names
categories_split.columns = category_colnames

# Display the first few rows of the `categories_split` DataFrame to verify the split
print("Categories split into separate columns:")
print(categories_split.head())


Categories split into separate columns:
0    related    request    offer    aid_related    medical_help  \
0  related-1  request-0  offer-0  aid_related-0  medical_help-0   
1  related-1  request-0  offer-0  aid_related-1  medical_help-0   
2  related-1  request-0  offer-0  aid_related-0  medical_help-0   
3  related-1  request-1  offer-0  aid_related-1  medical_help-0   
4  related-1  request-0  offer-0  aid_related-0  medical_help-0   

0    medical_products    search_and_rescue    security    military  \
0  medical_products-0  search_and_rescue-0  security-0  military-0   
1  medical_products-0  search_and_rescue-0  security-0  military-0   
2  medical_products-0  search_and_rescue-0  security-0  military-0   
3  medical_products-1  search_and_rescue-0  security-0  military-0   
4  medical_products-0  search_and_rescue-0  security-0  military-0   

0    child_alone       ...           aid_centers    other_infrastructure  \
0  child_alone-0       ...         aid_centers-0  other_infr

### 4. Convert category values to just numbers 0 or 1.
- Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. Convert the string to a numeric value.
- You can perform [normal string actions on Pandas Series](https://pandas.pydata.org/pandas-docs/stable/text.html#indexing-with-str), like indexing, by including `.str` after the Series. You may need to first convert the Series to be of type string, which you can do with `astype(str)`.

In [17]:
# Convert category values to just numbers 0 or 1
for column in categories_split:
    # Set each value to be the last character of the string
    categories_split[column] = categories_split[column].str[-1]
    # Convert column from string to numeric
    categories_split[column] = categories_split[column].astype(int)

# Display the first few rows of the `categories_split` DataFrame to verify the conversion
print("Categories converted to binary values:")
print(categories_split.head())


Categories converted to binary values:
0  related  request  offer  aid_related  medical_help  medical_products  \
0        1        0      0            0             0                 0   
1        1        0      0            1             0                 0   
2        1        0      0            0             0                 0   
3        1        1      0            1             0                 1   
4        1        0      0            0             0                 0   

0  search_and_rescue  security  military  child_alone      ...        \
0                  0         0         0            0      ...         
1                  0         0         0            0      ...         
2                  0         0         0            0      ...         
3                  0         0         0            0      ...         
4                  0         0         0            0      ...         

0  aid_centers  other_infrastructure  weather_related  floods  storm  fire  \

### 5. Replace `categories` column in `df` with new category columns.
- Drop the categories column from the df dataframe since it is no longer needed.
- Concatenate df and categories data frames.

In [18]:
# Assuming df and categories_split DataFrames are already defined from previous steps

# Drop the original 'categories' column from df
df = df.drop('categories', axis=1)

# Concatenate df with the new categories DataFrame
df = pd.concat([df, categories_split], axis=1)

# Display the first few rows of the updated df to verify the concatenation
print("Dataframe after replacing 'categories' column with new category columns:")
print(df.head())


Dataframe after replacing 'categories' column with new category columns:
   id                                            message  \
0   2  Weather update - a cold front from Cuba that c...   
1   7            Is the Hurricane over or is it not over   
2   8                    Looking for someone but no name   
3   9  UN reports Leogane 80-90 destroyed. Only Hospi...   
4  12  says: west side of Haiti, rest of the country ...   

                                            original   genre  related  \
0  Un front froid se retrouve sur Cuba ce matin. ...  direct        1   
1                 Cyclone nan fini osinon li pa fini  direct        1   
2  Patnm, di Maryani relem pou li banm nouvel li ...  direct        1   
3  UN reports Leogane 80-90 destroyed. Only Hospi...  direct        1   
4  facade ouest d Haiti et le reste du pays aujou...  direct        1   

   request  offer  aid_related  medical_help  medical_products      ...        \
0        0      0            0             0  

### 6. Remove duplicates.
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

In [19]:
# Check how many duplicates are in the dataset
num_duplicates = df.duplicated().sum()
print(f"Number of duplicates in the dataset: {num_duplicates}")


Number of duplicates in the dataset: 170


In [20]:
# Drop duplicates
df = df.drop_duplicates()

In [21]:
# Confirm duplicates were removed by checking again
num_duplicates_after = df.duplicated().sum()
print(f"Number of duplicates after removing them: {num_duplicates_after}")

Number of duplicates after removing them: 0


### 7. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library. Remember to import SQLAlchemy's `create_engine` in the first cell of this notebook to use it below.

In [23]:
def save_to_database(df, folder_path="Data", db_filename="DisasterResponse.db"):
    """
    Saves a DataFrame to an SQLite database.

    This function checks if the specified directory exists, creates it if not,
    and then saves the DataFrame to a specified SQLite database file within that directory.

    Args:
        df (pd.DataFrame): DataFrame to be saved.
        folder_path (str): Directory where the database file will be saved.
        db_filename (str): Name of the database file.

    Returns:
        None
    """
    # Ensure the target directory exists
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    # Define the full path to the database file
    db_path = os.path.join(folder_path, db_filename)

    # Create a SQLAlchemy engine for the SQLite database connection
    engine = create_engine(f'sqlite:///{db_path}')

    # Save the DataFrame to the specified SQLite database
    df.to_sql('DisasterResponse', engine, index=False, if_exists='replace')

### 8. Use this notebook to complete `process_data.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `process_data.py` in the classroom on the `Project Workspace IDE` coming later.